## Before running: DCA configuration
Use MT-VIKI monitor switch to switch between oscilloscope (#1) and BERT (#2)

1. Verify that you are using an #86105D plugin with 10G hardware filter
2. Verify that the J-BERT is outputting through PPG PRBS-7 signal at 25.78125 Gb/s
3. Turn on the correct source and turn off all other sources (eg. source 1, 2, etc.)
4. Verify that the source has its hardware filter set to 10.31250 Gb/s
5. On FlexDCA, File>Open Instrument Setup>setup_qsfp_sr4
6. Confirm presence of 100GE-SR4 mask test measurement
7. Set mask test source to the correct input (eg. source 1, 2, etc.)
8. Confirm presence of TDEC[OMA XP] measurement
9. Set TDEC source to the correct input (eg. source 1, 2, etc.)
10. Confirm presence of average power measurement
11. Set average power source to the correct input (eg. source 1, 2, etc.)

Note: to run a code block, have cursor focus on the block, then click "Run" in the toolbar

### Run the following once to connect to instruments

In [1]:
import os
import visa
import time
import datetime
import csv
import numpy as np

#test connection
rm = visa.ResourceManager()
#rm.list_resources() #lists available resources

#connecting to relevant resources
scope = rm.open_resource('GPIB0::7::INSTR')
thermostream = rm.open_resource('GPIB0::30::INSTR')
print('Instruments connected successfully')

Instruments connected successfully


### Run the following once to initialize functions

In [2]:
#function performs measurements at given temperatures
def config_temps(hot=70, amb=35, cold=0):
    thermostream.write('SETN 0') #selecting hot
    thermostream.write('SETP ' + repr(hot)) #setting hot
    
    thermostream.write('SETN 1') #selecting amb
    thermostream.write('SETP ' + repr(amb)) #setting amb
    
    thermostream.write('SETN 2') #selecting cold
    thermostream.write('SETP ' + repr(cold)) #setting cold

#function runs acquisition for specified number of seconds
def run_until(seconds,channel=1):
    scope.write('STOP') #stop acquisition
    scope.write('CDISplay') #clear display
    scope.write(':SYSTem:AUTOscale')
    scope.timeout = 10000 #sets timeout to 10 secs
    scope.query('*OPC?') #waits for commands to finish
    scope.write('RUN') #start acquisition
    scope.query('*OPC?') #waits for commands to finish
    time.sleep(seconds)
    scope.timeout = 2000 #resets timeout to 2 secs

#function updates recorded data into data array
def update_data(data, new_data):
    params = len(new_data)
    for i in range(params):
        data[i].append(new_data[i])

#function exports data to csv file
def export_data(data, headers, filename='qsfp_data'):
    headers = zip(*headers) #transposing headers
    data = zip(*data) #transposing data
    with open(data_dir + filename + '.csv', "w") as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerows(headers) #writing headers
        writer.writerows(data) #writing data
        
print('Functions initialized successfully')

Functions initialized successfully


### Run the following once to initialize measurement parameters

In [3]:
# configuring data table
headers = [['SN'], ['Channel'], ['Voltage'], ['Temp'], ['TDEC'], ['TxMM'], ['Time'], ['TDEC_P/F'], ['TxMM_P/F'], ['Device_P/F']]
data = [[] for i in range(len(headers))]

#setting data directory
data_dir = os.getcwd()[:33] + 'data\\' #The slice [:33] is tailored to the system directory - you will need to change it to slice the correct path
as_dir = 'auto_save' #auto_save used until the user intitalizes a data file

#setting image directory (Saved locally on DCA - Folders must already exist)
image_dir = 'D:\\User Files\\Screen Images\\Test Automation\\Eye Diagrams\\'

#Setting Thermostream Levels
hot = 70
amb = 35
cold = 0
temperatures = [hot,amb,cold]

config_temps(hot,amb,cold) #configuring temperatures
thermostream.write('SETN 1') #setting to ambient

print('Parameters initialized successfully')

Parameters initialized successfully


### Run to create local data file
Data is saved locally in C:\tester2\Test Automation\data

Data will not be saved if folder does not already exist

Note: screen images are saved automatically to the DCA in D:\User Files\Screen Images\Test Automation\Eye Diagrams

In [4]:
#export data to specified -16path
folder = 'AFBR-89CEDZ-DC1_Batch10EA_Test1\\'
filename = 'qsfp28_esr4_data_1'
as_dir = folder + filename
export_data(data,headers,as_dir)
print('Success!')

Success!


### Run once to record and preview data
1. Verify that 'source' is set to the correct value
2. Verify that 'serial_number' matches the current part
3. Verify that 'channel' matches the current channel
4. Verify that the head is down

In [25]:
source = 3 #this is the DCA source channel being used to collect data
channel = 2 #this is the current QSFP channel number

tdec_max = 4.2 #this is the maximum TDEC to be within spec
mm_min = 10 #this is the minimum TxMM to be within spec

serial_number = input('What is the current serial number? ') #queries user for device serial number
#channel = input('What is the current channel?') #queries user for device channel

if not input('\nStart testing? [Enter to confirm]') : pass #waits to begin testing until user confirms

#running test
for i in range(len(temperatures)): #iterating over temperatures in order hot-amb-cold
    
    #cycling temperature
    current_temp = temperatures[i]
    thermostream.write('SETN ' + repr(i)) #setting temperature
    print('Cycling to ' + repr(temperatures[i]))
    while(np.absolute(temperatures[i] - float(thermostream.query('TEMP?'))) != 0): pass #waiting for temperature to reach setpoint
    time.sleep(90) #waiting 90 seconds after temperature reaches setpoint
    print('Collecting data')
    
    #taking unfiltered eye data
    scope.write(':SYSTem:MODE EYE') #confirming the dca is in eye mode
    scope.write('CHANnel' + repr(source) + ':FILTer OFF') #confirming the filter is off
    run_until(15) #acquiring for 15 seconds
    scope.write(':DISK:SIMage:FNAMe "' + image_dir + serial_number + '_eye_unfiltered_chan' + repr(channel) + '_' + repr(current_temp) + 'C.png"') #writing screen image path
    scope.write(':DISK:SIMage:SAVE') #saving screen image
    mm = float(scope.query(':MEASure:MTESt:MARGin?')) #reading mask margin
    scope.query('*OPC?') #waiting for issued commands to terminate
    
    #taking filtered eye data
    scope.write('CHANnel' + repr(source) + ':FILTer ON') #confirming the filter is on
    run_until(15) #acquiring for 15 seconds
    scope.write(':DISK:SIMage:FNAMe "' + image_dir + serial_number + '_eye_filtered_chan' + repr(channel) + '_' + repr(current_temp) + 'C.png"')  #writing screen image path
    scope.write(':DISK:SIMage:SAVE') #saving screen image
    tdec = float(scope.query(':MEASure:EYE:TDEC?')) #reading TDEC
    scope.query('*OPC?') #waiting for issued commands to terminate
    
    scope.write('STOP') #stopping scope
    
    #printing data
    print('TDEC: ' + str(tdec)) #printing TDEC
    print('TxMM: ' + str(mm) + '\n') #printing mask margin
    
    #checking spec limits
    tdec_pf = 'Pass'
    mm_pf = 'Pass'
    o_pf = 'Pass'
    
    if tdec > tdec_max: tdec_pf = 'Fail' #checking tdec
    
    if mm < mm_min: mm_pf = 'Fail' #checking mask margin
    
    if tdec_pf == 'Fail' or mm_pf == 'Fail':
        o_pf = 'Fail'
        print('WARNING: Spec limits exceeded')
        
    
    now = datetime.datetime.now().strftime('%y-%m-%d %I:%M %p') #getting current time
    new_data=[serial_number, channel, 3.3, current_temp, tdec, tdec_pf, mm, mm_pf, o_pf, now] #formatting data
    update_data(data, new_data) #writing data to array

export_data(data,headers,filename=as_dir) #exporting data to file

thermostream.write('SETN 1') #resetting temperature to ambient

print('Testing complete!')

What is the current serial number? AF1805GL05W

Start testing? [Enter to confirm]
Cycling to 70
TDEC: 4.7
TxMM: 27.7


Cycling to 35
TDEC: 4.42
TxMM: 35.2


Cycling to 0
TDEC: 4.26
TxMM: 40.5


Testing complete!
